In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [2]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [6]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc3
/device:GPU:0
Num GPUs Available:  1


In [0]:
def Model(num_classes):
  layers = [
      tf.keras.layers.Dense(128, activation = tf.nn.relu),
      tf.keras.layers.Dense(64, activation = tf.nn.relu),
      tf.keras.layers.Dense(32, activation = tf.nn.relu),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, y_train.shape)
  print(X_test.shape,y_test.shape)

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=15)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=32, epochs=200,callbacks=[es])
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
def OpportunityDataSetAnalysis():
  X_train=np.load('data/Opportunity/Opportunity_train_X.npz')['arr_0']
  X_test=np.load('data/Opportunity/Opportunity_test_X.npz')['arr_0']
  Y_train=np.load('data/Opportunity/Opportunity_train_y.npz')['arr_0']
  Y_test=np.load('data/Opportunity/Opportunity_test_Y.npz')['arr_0']
  classes_number = Y_train.shape[1]
  Y_train = np.argmax(Y_train, axis=1)
  Y_test = np.argmax(Y_test, axis=1)
  return X_train,Y_train,X_test,Y_test,classes_number

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes):
  avg_acc = []
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, Y_train.shape)
  print(X_test.shape,Y_test.shape)
  X_train, X_Val, Y_train, Y_Val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

  model = Model(num_classes)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, Y_train, batch_size=32, epochs=200,callbacks=[es],validation_data=(X_Val,Y_Val))
  _,acc=model.evaluate(X_test,Y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracyOpportunityWithoutPCA(X_train,Y_train,X_test,Y_test,num_classes):
  avg_acc = []
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, Y_train.shape)
  print(X_test.shape,Y_test.shape)
  X_train, X_Val, Y_train, Y_Val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

  model = Model(num_classes)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, Y_train, batch_size=32, epochs=200,callbacks=[es],validation_data=(X_Val,Y_Val))
  _,acc=model.evaluate(X_test,Y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))

In [0]:
def RunOpportunity():
  X_train,Y_train,X_test,Y_test,num_classes=OpportunityDataSetAnalysis()
  TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes)

In [0]:
def RunOpportunityWithoutPCA():
  X_train,Y_train,X_test,Y_test,num_classes=OpportunityDataSetAnalysis()
  TrainAndReportAccuracyOpportunityWithPCA(X_train,Y_train,X_test,Y_test,num_classes)

In [32]:
RunOpportunityWithoutPCA()

(46495, 453) (46495,)
(9894, 453) (9894,)
Epoch 1/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.3312 - sparse_categorical_accuracy: 0.8772 - val_loss: 0.2537 - val_sparse_categorical_accuracy: 0.9068
Epoch 2/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.2197 - sparse_categorical_accuracy: 0.9186 - val_loss: 0.2162 - val_sparse_categorical_accuracy: 0.9192
Epoch 3/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1820 - sparse_categorical_accuracy: 0.9317 - val_loss: 0.2125 - val_sparse_categorical_accuracy: 0.9215
Epoch 4/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1602 - sparse_categorical_accuracy: 0.9401 - val_loss: 0.1953 - val_sparse_categorical_accuracy: 0.9263
Epoch 5/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1391 - sparse_categorical_accuracy: 0.9482 - val_loss: 0.2143 - val_sparse_categorical_accuracy: 0.9262
Epoch 6/200
1163/1163 [====================

In [30]:
RunOpportunity()

(46495, 118) (46495,)
(9894, 118) (9894,)
Epoch 1/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.8698 - val_loss: 0.2625 - val_sparse_categorical_accuracy: 0.9002
Epoch 2/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.2116 - sparse_categorical_accuracy: 0.9196 - val_loss: 0.2149 - val_sparse_categorical_accuracy: 0.9216
Epoch 3/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1719 - sparse_categorical_accuracy: 0.9352 - val_loss: 0.2071 - val_sparse_categorical_accuracy: 0.9287
Epoch 4/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1431 - sparse_categorical_accuracy: 0.9469 - val_loss: 0.2152 - val_sparse_categorical_accuracy: 0.9225
Epoch 5/200
1163/1163 [==============================] - 4s 3ms/step - loss: 0.1248 - sparse_categorical_accuracy: 0.9522 - val_loss: 0.2105 - val_sparse_categorical_accuracy: 0.9270
Epoch 6/200
1163/1163 [====================

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/200
54/54 [==============================] - 0s 2ms/step - loss: 1.4026 - sparse_categorical_accuracy: 0.6710
Epoch 2/200
54/54 [==============================] - 0s 2ms/step - loss: 0.2168 - sparse_categorical_accuracy: 0.9626
Epoch 3/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0657 - sparse_categorical_accuracy: 0.9924
Epoch 4/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0357 - sparse_categorical_accuracy: 0.9942
Epoch 5/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0215 - sparse_categorical_accuracy: 0.9977
Epoch 6/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0141 - sparse_categorical_accuracy: 0.9982
Epoch 7/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0106 - sparse_categorical_accuracy: 0.9977
Epoch 8/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0075 - sparse_categoric

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/200
54/54 [==============================] - 0s 2ms/step - loss: 1.4032 - sparse_categorical_accuracy: 0.6143
Epoch 2/200
54/54 [==============================] - 0s 2ms/step - loss: 0.2813 - sparse_categorical_accuracy: 0.9433
Epoch 3/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0824 - sparse_categorical_accuracy: 0.9842
Epoch 4/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0404 - sparse_categorical_accuracy: 0.9953
Epoch 5/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0236 - sparse_categorical_accuracy: 0.9982
Epoch 6/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0142 - sparse_categorical_accuracy: 0.9994
Epoch 7/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0113 - sparse_categorical_accuracy: 0.9982
Epoch 8/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0074 - sparse_categoric

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(894, 22) (894,)
(441, 22) (441,)
Epoch 1/200
28/28 [==============================] - 0s 2ms/step - loss: 1.8362 - sparse_categorical_accuracy: 0.4989
Epoch 2/200
28/28 [==============================] - 0s 2ms/step - loss: 0.8041 - sparse_categorical_accuracy: 0.8054
Epoch 3/200
28/28 [==============================] - 0s 2ms/step - loss: 0.3373 - sparse_categorical_accuracy: 0.9418
Epoch 4/200
28/28 [==============================] - 0s 2ms/step - loss: 0.1589 - sparse_categorical_accuracy: 0.9787
Epoch 5/200
28/28 [==============================] - 0s 2ms/step - loss: 0.0878 - sparse_categorical_accuracy: 0.9888
Epoch 6/200
28/28 [==============================] - 0s 2ms/step - loss: 0.0598 - sparse_categorical_accuracy: 0.9922
Epoch 7/200
28/28 [==============================] - 0s 2ms/step - loss: 0.0486 - sparse_categorical_accuracy: 0.9888
Epoch 8/200
28/28 [==============================] - 0s 2ms/step - loss: 0.0332 - sparse_categorical

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(1711, 22) (1711,)
(844, 22) (844,)
Epoch 1/200
54/54 [==============================] - 0s 2ms/step - loss: 1.4134 - sparse_categorical_accuracy: 0.6131
Epoch 2/200
54/54 [==============================] - 0s 2ms/step - loss: 0.2529 - sparse_categorical_accuracy: 0.9585
Epoch 3/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0837 - sparse_categorical_accuracy: 0.9842
Epoch 4/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0428 - sparse_categorical_accuracy: 0.9947
Epoch 5/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0269 - sparse_categorical_accuracy: 0.9971
Epoch 6/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0178 - sparse_categorical_accuracy: 0.9965
Epoch 7/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0120 - sparse_categorical_accuracy: 0.9988
Epoch 8/200
54/54 [==============================] - 0s 2ms/step - loss: 0.0092 - sparse_categoric

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/200
206/206 [==============================] - 0s 2ms/step - loss: 1.1367 - sparse_categorical_accuracy: 0.6197
Epoch 2/200
206/206 [==============================] - 0s 2ms/step - loss: 0.5629 - sparse_categorical_accuracy: 0.7818
Epoch 3/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4805 - sparse_categorical_accuracy: 0.8110
Epoch 4/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4289 - sparse_categorical_accuracy: 0.8300
Epoch 5/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3890 - sparse_categorical_accuracy: 0.8406
Epoch 6/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3583 - sparse_categorical_accuracy: 0.8557
Epoch 7/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3448 - sparse_categorical_accuracy: 0.8590
Epoch 8/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3285 - s

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/200
206/206 [==============================] - 0s 2ms/step - loss: 1.0872 - sparse_categorical_accuracy: 0.6431
Epoch 2/200
206/206 [==============================] - 0s 2ms/step - loss: 0.5743 - sparse_categorical_accuracy: 0.7716
Epoch 3/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4955 - sparse_categorical_accuracy: 0.8010
Epoch 4/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4418 - sparse_categorical_accuracy: 0.8235
Epoch 5/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4076 - sparse_categorical_accuracy: 0.8374
Epoch 6/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3812 - sparse_categorical_accuracy: 0.8453
Epoch 7/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3593 - sparse_categorical_accuracy: 0.8538
Epoch 8/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3365 - s

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(3375, 7) (3375,)
(1663, 7) (1663,)
Epoch 1/200
106/106 [==============================] - 0s 2ms/step - loss: 1.4001 - sparse_categorical_accuracy: 0.5932
Epoch 2/200
106/106 [==============================] - 0s 2ms/step - loss: 0.7055 - sparse_categorical_accuracy: 0.7090
Epoch 3/200
106/106 [==============================] - 0s 2ms/step - loss: 0.6286 - sparse_categorical_accuracy: 0.7425
Epoch 4/200
106/106 [==============================] - 0s 2ms/step - loss: 0.5699 - sparse_categorical_accuracy: 0.7721
Epoch 5/200
106/106 [==============================] - 0s 2ms/step - loss: 0.5295 - sparse_categorical_accuracy: 0.7920
Epoch 6/200
106/106 [==============================] - 0s 2ms/step - loss: 0.4876 - sparse_categorical_accuracy: 0.8127
Epoch 7/200
106/106 [==============================] - 0s 2ms/step - loss: 0.4713 - sparse_categorical_accuracy: 0.8151
Epoch 8/200
106/106 [==============================] - 0s 2ms/step - loss: 0.4475 - s

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(6582, 7) (6582,)
(3242, 7) (3242,)
Epoch 1/200
206/206 [==============================] - 0s 2ms/step - loss: 1.0522 - sparse_categorical_accuracy: 0.6618
Epoch 2/200
206/206 [==============================] - 0s 2ms/step - loss: 0.5554 - sparse_categorical_accuracy: 0.7799
Epoch 3/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4732 - sparse_categorical_accuracy: 0.8162
Epoch 4/200
206/206 [==============================] - 0s 2ms/step - loss: 0.4297 - sparse_categorical_accuracy: 0.8318
Epoch 5/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3881 - sparse_categorical_accuracy: 0.8493
Epoch 6/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3701 - sparse_categorical_accuracy: 0.8548
Epoch 7/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3487 - sparse_categorical_accuracy: 0.8608
Epoch 8/200
206/206 [==============================] - 0s 2ms/step - loss: 0.3259 - s

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
Epoch 1/200
82/82 [==============================] - 0s 2ms/step - loss: 1.6900 - sparse_categorical_accuracy: 0.4987
Epoch 2/200
82/82 [==============================] - 0s 2ms/step - loss: 1.1924 - sparse_categorical_accuracy: 0.5573
Epoch 3/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0875 - sparse_categorical_accuracy: 0.5866
Epoch 4/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0219 - sparse_categorical_accuracy: 0.6066
Epoch 5/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0106 - sparse_categorical_accuracy: 0.6167
Epoch 6/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9712 - sparse_categorical_accuracy: 0.6348
Epoch 7/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9223 - sparse_categorical_accuracy: 0.6491
Epoch 8/200
82/82 [==============================] - 0s 2ms/step - loss: 0.8978 - sparse_categorical

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(2593, 5) (2593,)
(1278, 5) (1278,)
Epoch 1/200
82/82 [==============================] - 0s 2ms/step - loss: 1.7073 - sparse_categorical_accuracy: 0.4597
Epoch 2/200
82/82 [==============================] - 0s 2ms/step - loss: 1.1787 - sparse_categorical_accuracy: 0.5511
Epoch 3/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0613 - sparse_categorical_accuracy: 0.6070
Epoch 4/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0090 - sparse_categorical_accuracy: 0.6155
Epoch 5/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9660 - sparse_categorical_accuracy: 0.6487
Epoch 6/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9394 - sparse_categorical_accuracy: 0.6437
Epoch 7/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9047 - sparse_categorical_accuracy: 0.6494
Epoch 8/200
82/82 [==============================] - 0s 2ms/step - loss: 0.8923 - sparse_categorical

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(2599, 5) (2599,)
(1281, 5) (1281,)
Epoch 1/200
82/82 [==============================] - 0s 2ms/step - loss: 1.7513 - sparse_categorical_accuracy: 0.4456
Epoch 2/200
82/82 [==============================] - 0s 2ms/step - loss: 1.2439 - sparse_categorical_accuracy: 0.5518
Epoch 3/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0969 - sparse_categorical_accuracy: 0.5810
Epoch 4/200
82/82 [==============================] - 0s 2ms/step - loss: 1.0245 - sparse_categorical_accuracy: 0.6199
Epoch 5/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9772 - sparse_categorical_accuracy: 0.6341
Epoch 6/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9474 - sparse_categorical_accuracy: 0.6479
Epoch 7/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9219 - sparse_categorical_accuracy: 0.6472
Epoch 8/200
82/82 [==============================] - 0s 2ms/step - loss: 0.9017 - sparse_categorical

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(1437, 5) (1437,)
(709, 5) (709,)
Epoch 1/200
45/45 [==============================] - 0s 2ms/step - loss: 2.0847 - sparse_categorical_accuracy: 0.3702
Epoch 2/200
45/45 [==============================] - 0s 2ms/step - loss: 1.4296 - sparse_categorical_accuracy: 0.5052
Epoch 3/200
45/45 [==============================] - 0s 2ms/step - loss: 1.1958 - sparse_categorical_accuracy: 0.5539
Epoch 4/200
45/45 [==============================] - 0s 2ms/step - loss: 1.0999 - sparse_categorical_accuracy: 0.5832
Epoch 5/200
45/45 [==============================] - 0s 2ms/step - loss: 1.0445 - sparse_categorical_accuracy: 0.5971
Epoch 6/200
45/45 [==============================] - 0s 2ms/step - loss: 1.0005 - sparse_categorical_accuracy: 0.6089
Epoch 7/200
45/45 [==============================] - 0s 2ms/step - loss: 0.9695 - sparse_categorical_accuracy: 0.6186
Epoch 8/200
45/45 [==============================] - 0s 2ms/step - loss: 0.9487 - sparse_categorical_a

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/200
437/437 [==============================] - 1s 2ms/step - loss: 0.5909 - sparse_categorical_accuracy: 0.7979
Epoch 2/200
437/437 [==============================] - 1s 2ms/step - loss: 0.4048 - sparse_categorical_accuracy: 0.8507
Epoch 3/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3592 - sparse_categorical_accuracy: 0.8668
Epoch 4/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3331 - sparse_categorical_accuracy: 0.8759
Epoch 5/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3075 - sparse_categorical_accuracy: 0.8868
Epoch 6/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2940 - sparse_categorical_accuracy: 0.8881
Epoch 7/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2810 - sparse_categorical_accuracy: 0.8950
Epoch 8/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2632 - s

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/200
437/437 [==============================] - 1s 2ms/step - loss: 0.5940 - sparse_categorical_accuracy: 0.7999
Epoch 2/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3975 - sparse_categorical_accuracy: 0.8556
Epoch 3/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3612 - sparse_categorical_accuracy: 0.8678
Epoch 4/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3328 - sparse_categorical_accuracy: 0.8768
Epoch 5/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3099 - sparse_categorical_accuracy: 0.8858
Epoch 6/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2928 - sparse_categorical_accuracy: 0.8915
Epoch 7/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2743 - sparse_categorical_accuracy: 0.8993
Epoch 8/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2588 - s

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(13966, 5) (13966,)
(6880, 5) (6880,)
Epoch 1/200
437/437 [==============================] - 1s 2ms/step - loss: 0.5882 - sparse_categorical_accuracy: 0.7946
Epoch 2/200
437/437 [==============================] - 1s 2ms/step - loss: 0.4046 - sparse_categorical_accuracy: 0.8493
Epoch 3/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3592 - sparse_categorical_accuracy: 0.8683
Epoch 4/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3309 - sparse_categorical_accuracy: 0.8779
Epoch 5/200
437/437 [==============================] - 1s 2ms/step - loss: 0.3089 - sparse_categorical_accuracy: 0.8831
Epoch 6/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2930 - sparse_categorical_accuracy: 0.8927
Epoch 7/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2759 - sparse_categorical_accuracy: 0.8990
Epoch 8/200
437/437 [==============================] - 1s 2ms/step - loss: 0.2661 - s

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(7045, 5) (7045,)
(3471, 5) (3471,)
Epoch 1/200
221/221 [==============================] - 0s 2ms/step - loss: 0.7181 - sparse_categorical_accuracy: 0.7719
Epoch 2/200
221/221 [==============================] - 0s 2ms/step - loss: 0.4691 - sparse_categorical_accuracy: 0.8248
Epoch 3/200
221/221 [==============================] - 0s 2ms/step - loss: 0.4235 - sparse_categorical_accuracy: 0.8468
Epoch 4/200
221/221 [==============================] - 0s 2ms/step - loss: 0.3990 - sparse_categorical_accuracy: 0.8529
Epoch 5/200
221/221 [==============================] - 0s 2ms/step - loss: 0.3807 - sparse_categorical_accuracy: 0.8589
Epoch 6/200
221/221 [==============================] - 0s 2ms/step - loss: 0.3629 - sparse_categorical_accuracy: 0.8661
Epoch 7/200
221/221 [==============================] - 0s 2ms/step - loss: 0.3488 - sparse_categorical_accuracy: 0.8718
Epoch 8/200
221/221 [==============================] - 0s 2ms/step - loss: 0.3406 - spa

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 1.5422 - sparse_categorical_accuracy: 0.3758
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 1.1564 - sparse_categorical_accuracy: 0.5795
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 0.9154 - sparse_categorical_accuracy: 0.6754
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7896 - sparse_categorical_accuracy: 0.7122
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6952 - sparse_categorical_accuracy: 0.7332
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6448 - sparse_categorical_accuracy: 0.7477
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6009 - sparse_categorical_accuracy: 0.7700
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5699 - sparse_categoric

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 1.4948 - sparse_categorical_accuracy: 0.4625
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 1.0376 - sparse_categorical_accuracy: 0.6544
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 0.8295 - sparse_categorical_accuracy: 0.6925
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 0.7215 - sparse_categorical_accuracy: 0.7122
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6489 - sparse_categorical_accuracy: 0.7398
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6118 - sparse_categorical_accuracy: 0.7582
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5713 - sparse_categorical_accuracy: 0.7687
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5505 - sparse_categoric

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(412, 7) (412,)
(204, 7) (204,)
Epoch 1/200
13/13 [==============================] - 0s 2ms/step - loss: 1.6138 - sparse_categorical_accuracy: 0.4490
Epoch 2/200
13/13 [==============================] - 0s 2ms/step - loss: 1.3229 - sparse_categorical_accuracy: 0.5607
Epoch 3/200
13/13 [==============================] - 0s 2ms/step - loss: 1.1213 - sparse_categorical_accuracy: 0.5898
Epoch 4/200
13/13 [==============================] - 0s 2ms/step - loss: 0.9844 - sparse_categorical_accuracy: 0.6189
Epoch 5/200
13/13 [==============================] - 0s 2ms/step - loss: 0.8876 - sparse_categorical_accuracy: 0.6675
Epoch 6/200
13/13 [==============================] - 0s 2ms/step - loss: 0.8080 - sparse_categorical_accuracy: 0.6893
Epoch 7/200
13/13 [==============================] - 0s 2ms/step - loss: 0.7538 - sparse_categorical_accuracy: 0.7039
Epoch 8/200
13/13 [==============================] - 0s 2ms/step - loss: 0.7006 - sparse_categoric

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(761, 8) (761,)
(376, 8) (376,)
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 1.4768 - sparse_categorical_accuracy: 0.5112
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 1.0425 - sparse_categorical_accuracy: 0.6491
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 0.8384 - sparse_categorical_accuracy: 0.7057
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 0.7549 - sparse_categorical_accuracy: 0.7070
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6826 - sparse_categorical_accuracy: 0.7254
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6312 - sparse_categorical_accuracy: 0.7398
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 0.6052 - sparse_categorical_accuracy: 0.7595
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5775 - sparse_categoric

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/200
79/79 [==============================] - 0s 2ms/step - loss: 2.7869 - sparse_categorical_accuracy: 0.1730
Epoch 2/200
79/79 [==============================] - 0s 2ms/step - loss: 2.2165 - sparse_categorical_accuracy: 0.3353
Epoch 3/200
79/79 [==============================] - 0s 2ms/step - loss: 1.8602 - sparse_categorical_accuracy: 0.4224
Epoch 4/200
79/79 [==============================] - 0s 2ms/step - loss: 1.6620 - sparse_categorical_accuracy: 0.4739
Epoch 5/200
79/79 [==============================] - 0s 2ms/step - loss: 1.5256 - sparse_categorical_accuracy: 0.5131
Epoch 6/200
79/79 [==============================] - 0s 2ms/step - loss: 1.4311 - sparse_categorical_accuracy: 0.5392
Epoch 7/200
79/79 [==============================] - 0s 2ms/step - loss: 1.3442 - sparse_categorical_accuracy: 0.5645
Epoch 8/200
79/79 [==============================] - 0s 2ms/step - loss: 1.2901 - sparse_ca

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/200
79/79 [==============================] - 0s 2ms/step - loss: 2.8161 - sparse_categorical_accuracy: 0.1623
Epoch 2/200
79/79 [==============================] - 0s 2ms/step - loss: 2.2350 - sparse_categorical_accuracy: 0.3361
Epoch 3/200
79/79 [==============================] - 0s 2ms/step - loss: 1.8621 - sparse_categorical_accuracy: 0.4173
Epoch 4/200
79/79 [==============================] - 0s 2ms/step - loss: 1.6577 - sparse_categorical_accuracy: 0.4699
Epoch 5/200
79/79 [==============================] - 0s 2ms/step - loss: 1.5297 - sparse_categorical_accuracy: 0.5036
Epoch 6/200
79/79 [==============================] - 0s 2ms/step - loss: 1.4449 - sparse_categorical_accuracy: 0.5337
Epoch 7/200
79/79 [==============================] - 0s 2ms/step - loss: 1.3685 - sparse_categorical_accuracy: 0.5614
Epoch 8/200
79/79 [==============================] - 0s 2ms/step - loss: 1.3121 - sparse_ca

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(2526, 10) (2526,)
(1245, 10) (1245,)
Epoch 1/200
79/79 [==============================] - 0s 2ms/step - loss: 2.8604 - sparse_categorical_accuracy: 0.1564
Epoch 2/200
79/79 [==============================] - 0s 2ms/step - loss: 2.2784 - sparse_categorical_accuracy: 0.3250
Epoch 3/200
79/79 [==============================] - 0s 2ms/step - loss: 1.8916 - sparse_categorical_accuracy: 0.4101
Epoch 4/200
79/79 [==============================] - 0s 2ms/step - loss: 1.6771 - sparse_categorical_accuracy: 0.4759
Epoch 5/200
79/79 [==============================] - 0s 2ms/step - loss: 1.5409 - sparse_categorical_accuracy: 0.5107
Epoch 6/200
79/79 [==============================] - 0s 2ms/step - loss: 1.4501 - sparse_categorical_accuracy: 0.5432
Epoch 7/200
79/79 [==============================] - 0s 2ms/step - loss: 1.3705 - sparse_categorical_accuracy: 0.5610
Epoch 8/200
79/79 [==============================] - 0s 2ms/step - loss: 1.3060 - sparse_ca

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(1372, 10) (1372,)
(676, 10) (676,)
Epoch 1/200
43/43 [==============================] - 0s 2ms/step - loss: 2.9574 - sparse_categorical_accuracy: 0.0860
Epoch 2/200
43/43 [==============================] - 0s 2ms/step - loss: 2.7105 - sparse_categorical_accuracy: 0.2034
Epoch 3/200
43/43 [==============================] - 0s 2ms/step - loss: 2.3609 - sparse_categorical_accuracy: 0.3098
Epoch 4/200
43/43 [==============================] - 0s 2ms/step - loss: 2.0180 - sparse_categorical_accuracy: 0.3783
Epoch 5/200
43/43 [==============================] - 0s 2ms/step - loss: 1.7926 - sparse_categorical_accuracy: 0.4366
Epoch 6/200
43/43 [==============================] - 0s 2ms/step - loss: 1.6329 - sparse_categorical_accuracy: 0.4774
Epoch 7/200
43/43 [==============================] - 0s 2ms/step - loss: 1.5198 - sparse_categorical_accuracy: 0.5190
Epoch 8/200
43/43 [==============================] - 0s 2ms/step - loss: 1.4224 - sparse_cate